In [1]:
import nb_setup 
import importlib
importlib.reload(nb_setup)
nb_setup.init()

Added project root to sys.path: /Users/falcon/Developments/act/backtesting-playground/trading
Changed current working directory to: /Users/falcon/Developments/act/backtesting-playground/trading
Initialized project with base directory: /Users/falcon/Developments/act/backtesting-playground/trading


In [7]:
import numpy as np
import pandas as pd

df = pd.read_csv("data/all.csv")
df.shape

(21336, 13)

In [50]:
import numpy as np
import pandas as pd


def rank_column(df, column, ascending=False):
    _df = df.copy()
    col_name = f"rank_{column}"
    if col_name not in _df.columns:
        _df[col_name] = np.nan
    _df.loc[_df[col_name].isna(), col_name] = _df.groupby("start_time")[column].rank(
        ascending=ascending
    )
    return _df


def remove_volatile_coins(df: pd.DataFrame, n=5):
    """
    remove the most volatile coins, in terms of annualized standard deviation,
    """
    _df = df.copy()
    _df.loc[:, "pct_change"] = _df.groupby(["symbol"])["close"].pct_change() * 100
    rank_volatility = _df.groupby(["symbol"])["pct_change"].std() * np.sqrt(252)
    rank_volatility.sort_values(ascending=False, inplace=True)
    # print removed coins, print left coins
    display(rank_volatility)
    display(rank_volatility.head(n).index)
    _df = _df.loc[~_df["symbol"].isin(rank_volatility.head(n).index)].drop(columns="pct_change")
    return _df

In [52]:
_ = remove_volatile_coins(df, 5)
display(_.shape)
_.head()

symbol
WIF       106.227956
kFLOKI     90.428015
kPEPE      83.890005
kBONK      77.602031
kSHIB      66.807034
ONDO       66.684182
UNI        62.949357
FET        61.250304
JTO        61.076647
AR         60.828825
WLD        60.397364
JUP        59.608209
MAVIA      57.557368
PENDLE     56.931187
FTM        56.045259
PIXEL      54.352342
RNDR       53.626087
NEAR       52.441497
PYTH       52.225593
DOGE       51.980774
HBAR       51.638348
BCH        50.539765
AI         50.218730
NEO        49.928448
RUNE       49.763885
CFX        49.651106
ORDI       49.434161
SEI        47.609149
STRK       47.539106
ENS        47.496079
INJ        46.953070
TON        46.816098
TIA        46.529134
STX        45.842220
APT        45.803406
FIL        45.310114
SUI        43.417733
AVAX       42.357962
BLUR       41.982650
OP         41.316385
APE        40.857720
SOL        40.750205
LDO        40.432006
CAKE       37.786555
ARB        37.132508
MKR        36.753550
ADA        36.249737
DOT   

Index(['WIF', 'kFLOKI', 'kPEPE', 'kBONK', 'kSHIB'], dtype='object', name='symbol')

(19431, 13)

,symbol,datetime,start_time,end_time,interval,number_of_trades,close,high,low,open,volume,tic,toc
0,BTC,2024-02-21-16:00:00,2024-02-21-16:00:00,2024-02-21-19:59:59,4h,32541,51052.0,51436.0,50638.0,51160.0,978.55598,1708531200000,1708545599999
1,BTC,2024-02-21-20:00:00,2024-02-21-20:00:00,2024-02-21-23:59:59,4h,33938,51909.0,52000.0,50862.0,51052.0,1273.72087,1708545600000,1708559999999
2,BTC,2024-02-22-00:00:00,2024-02-22-00:00:00,2024-02-22-03:59:59,4h,25035,51595.0,51911.0,51161.0,51910.0,743.73055,1708560000000,1708574399999
3,BTC,2024-02-22-04:00:00,2024-02-22-04:00:00,2024-02-22-07:59:59,4h,18721,51866.0,51961.0,51365.0,51595.0,618.39497,1708574400000,1708588799999
4,BTC,2024-02-22-08:00:00,2024-02-22-08:00:00,2024-02-22-11:59:59,4h,16534,51621.0,52054.0,51520.0,51867.0,709.76714,1708588800000,1708603199999


In [ ]:
def calculate_normalized_area_under_ema200(df):
    pass
